In [ ]:
"""
Guilherme Araújo Mendes de Souza - 156437
UNIFESP - ICT
IA

Atividade 10

Fontes:
https://www.geeksforgeeks.org/exploratory-data-analysis-on-iris-dataset/
https://www.youtube.com/watch?v=y0NHzMEanIU
"""

'\nGuilherme Araújo Mendes de Souza - 156437\nUNIFESP - ICT\nIA\n\nAtividade 10\n\nFontes:\nhttps://www.geeksforgeeks.org/exploratory-data-analysis-on-iris-dataset/\nhttps://www.youtube.com/watch?v=y0NHzMEanIU\n'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Algoritmo KNN
def distancia_euclidiana(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def prever(X_treino, y_treino, x_teste, k):
    distancias = [distancia_euclidiana(x_teste, x) for x in X_treino]
    k_indices = np.argsort(distancias)[:k]
    k_rotulos_mais_comuns = [y_treino[i] for i in k_indices]
    mais_comum = np.bincount(k_rotulos_mais_comuns).argmax()
    return mais_comum

def vizinhos(X_treino, y_treino, X_teste, k):
    previsoes = [prever(X_treino, y_treino, x, k) for x in X_teste]
    return previsoes

In [ ]:
#Algoritmo Naive Bayes
def treinar_naive_bayes(X, y):
    classes = np.unique(y)
    num_classes = len(classes)
    probs_classe = {}
    probs_palavra = {}

    for c in classes:
        probs_classe[c] = np.sum(y == c) / len(y)
        indices_classe = np.where(y == c)
        palavras_na_classe = [X[i].split() for i in indices_classe[0]]
        palavras_na_classe = [palavra for sublist in palavras_na_classe for palavra in sublist]
        probs_palavra[c] = {}
        total_palavras = len(palavras_na_classe)

        for palavra in palavras_na_classe:
            probs_palavra[c][palavra] = (palavras_na_classe.count(palavra) + 1) / (total_palavras + len(set(palavras_na_classe)))

    return probs_classe, probs_palavra

def prever_naive_bayes(texto, probs_classe, probs_palavra):
    melhor_classe = None
    max_probabilidade = -1

    for c, prob in probs_classe.items():
        palavras = texto.split()
        prob_c = np.log(prob)
        for palavra in palavras:
            if palavra in probs_palavra[c]:
                prob_c += np.log(probs_palavra[c][palavra])
        if prob_c > max_probabilidade:
            max_probabilidade = prob_c
            melhor_classe = c

    return melhor_classe

In [ ]:
#Algoritmo Hunt

def ganho_informacao(dados, rótulos, índice_característica):
    valores_únicos = np.unique(dados[:, índice_característica])
    entropia_total = 0

    for valor in valores_únicos:
        subset = dados[dados[:, índice_característica] == valor]
        probabilidade = len(subset) / len(dados)
        entropia = calcula_entropia(subset, rótulos)
        entropia_total += probabilidade * entropia

    entropia_atributo = calcula_entropia(dados, rótulos)
    ganho = entropia_atributo - entropia_total

    return ganho

def calcula_entropia(dados, rótulos):
    entropia = 0
    classes = np.unique(rótulos)

    for classe in classes:
        p = len(rótulos[rótulos == classe]) / len(rótulos)
        entropia -= p * np.log2(p)

    return entropia

def arvore(dados, rótulos, características):
    if len(np.unique(rótulos)) == 1:
        return np.unique(rótulos)[0]
    elif len(características) == 0:
        return np.argmax(np.bincount(rótulos))

    ganhos = []
    for índice_característica in características:
        ganhos.append(ganho_informacao(dados, rótulos, índice_característica))

    melhor_característica = características[np.argmax(ganhos)]
    árvore = {melhor_característica: {}}
    características_restantes = [x for x in características if x != melhor_característica]

    for valor in np.unique(dados[:, melhor_característica]):
        subset = dados[dados[:, melhor_característica] == valor]
        subrótulos = rótulos[dados[:, melhor_característica] == valor]
        árvore[melhor_característica][valor] = arvore(subset, subrótulos, características_restantes)

    return árvore

In [ ]:
df = pd.read_csv ('/content/Iris.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
sns.countplot(x='Species', data=df, )
plt.show()

In [ ]:
sns.scatterplot(x='SepalLengthCm', y='SepalWidthCm',
                hue='Species', data=df, )

plt.legend(bbox_to_anchor=(1, 1), loc=2)

plt.show()

In [ ]:
sns.pairplot(df.drop(['Id'], axis = 1),
             hue='Species', height=2)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10,10))

axes[0,0].set_title("Sepal Length")
axes[0,0].hist(df['SepalLengthCm'], bins=7)

axes[0,1].set_title("Sepal Width")
axes[0,1].hist(df['SepalWidthCm'], bins=5);

axes[1,0].set_title("Petal Length")
axes[1,0].hist(df['PetalLengthCm'], bins=6);

axes[1,1].set_title("Petal Width")
axes[1,1].hist(df['PetalWidthCm'], bins=6);

In [ ]:
plot = sns.FacetGrid(df, hue="Species")
plot.map(sns.distplot, "SepalLengthCm").add_legend()

plot = sns.FacetGrid(df, hue="Species")
plot.map(sns.distplot, "SepalWidthCm").add_legend()

plot = sns.FacetGrid(df, hue="Species")
plot.map(sns.distplot, "PetalLengthCm").add_legend()

plot = sns.FacetGrid(df, hue="Species")
plot.map(sns.distplot, "PetalWidthCm").add_legend()

plt.show()

In [ ]:
df.corr(method='pearson')

In [ ]:
sns.heatmap(df.corr(method='pearson').drop(
  ['Id'], axis=1).drop(['Id'], axis=0),
            annot = True);

plt.show()

In [ ]:
def graph(y):
    sns.boxplot(x="Species", y=y, data=df)

plt.figure(figsize=(10,10))

plt.subplot(221)
graph('SepalLengthCm')

plt.subplot(222)
graph('SepalWidthCm')

plt.subplot(223)
graph('PetalLengthCm')

plt.subplot(224)
graph('PetalWidthCm')

plt.show()

In [ ]:
sns.boxplot(x='SepalWidthCm', data=df)

In [ ]:
Q1 = np.percentile(df['SepalWidthCm'], 25,
                interpolation = 'midpoint')

Q3 = np.percentile(df['SepalWidthCm'], 75,
                interpolation = 'midpoint')
IQR = Q3 - Q1

print("Old Shape: ", df.shape)

upper = np.where(df['SepalWidthCm'] >= (Q3+1.5*IQR))

lower = np.where(df['SepalWidthCm'] <= (Q1-1.5*IQR))

df.drop(upper[0], inplace = True)
df.drop(lower[0], inplace = True)

print("New Shape: ", df.shape)

sns.boxplot(x='SepalWidthCm', data=df)

In [ ]:
#Separando 70% dos dados para treino e 30% para teste
from sklearn.model_selection import train_test_split

X = df[["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = df["Species"]

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score

k = 3

y_pred_knn = []

for i in X_teste.values:
  classe = vizinhos(X_treino.values, y_treino, i, k)
  y_pred_knn.append(classe)

acuracia_knn = accuracy_score(y_teste, y_pred_knn)
print("KNN:", acuracia_knn)

In [ ]:
probs_classe, probs_palavra = treinar_naive_bayes(X_treino.values, y_treino)
y_pred_bayes = [prever_naive_bayes(texto, probs_classe, probs_palavra) for texto in X_teste]

acuracia_bayes = accuracy_score(y_teste, y_pred_bayes)
print("Naive Bayes:", acuracia_bayes)

In [ ]:
arvore_decisão = arvore(X_treino.values, y_treino, X_teste.values)
print(arvore_decisão)